# DistMult 实践

在这个演示中，我们使用DistMUlt([论文链接](http://proceedings.mlr.press/v48/trouillon16.pdf))对示例中文知识图谱进行链接预测，从而达到补全知识图谱的目的。

希望在这个demo中帮助大家了解知识图谱表示学习的作用原理和机制。

本demo建议使用python3运行。

## 数据集
这个示例中，我们使用的是表示学习模型做知识图谱链接预测常用的benchmark数据集 FB15k-237：

| #Ent | #Rel | # Train | #Test | #Valid |
| --- | --- | --- | --- | --- |
| 14,541| 237| 272,115| 17,535| 20,466 |


### DistMult 原理回顾
DistMult将每个实体表示为一个向量，每个关系都表示成一个矩阵，
并假设对于一个存在在知识图谱中的三元组$(h,r,t)$, 
$h, r, t$的向量表示$\mathbf{h}, \mathbf{M}_r, \mathbf{t}$满足：
$$\mathbf{h}\mathbf{M}_r = \mathbf{t}$$


对于每个正确的三元组的优化目标是：
$$\mathbf{h} \mathbf{M}_r \approx \mathbf{t}$$
DistMult采用点积来衡量两个向量的相似度，所以对于一个三元组的评分函数为：
$$f_r(h,t) = \mathbf{h}\mathbf{M}_r \mathbf{t} $$
对于正样本评分较高，负样本评分较低

DistMult的损失函数：
$$ L = \sum_{(h,r,t)\in S} \sum_{(h^\prime, r^\prime, t^\prime) \in S^\prime} max(0, f_{r^\prime} (h^\prime, t^\prime)  + \gamma - f_r(h,t)) $$
其中$S$是所有正样本的集合，$S^\prime$是所有负样本的集合，对于一个正样本$(h,r,t)$负样本通过随机替换$h$或$t$得到， $\gamma$表示间隔，是一个超参。

根据DistMult原文的实验结果，当关系的表示设为对角阵时链接预测效果较好，本示例也采取了对角阵的表示方式，示例中采用了如下loss：
$$ L = \frac{1}{len(S)}\sum_{(h,r,t)\in \{S, S^\prime\}}log(e^{(- f_r(h,t)*lable)} + 1)$$
其中正样本的lable为1，负样本的lable为-1.



### 代码实践

In [1]:
import tensorflow as tf 
import time 
import argparse
import random
import numpy as np 
import os.path
import math
import timeit
from multiprocessing import JoinableQueue, Queue, Process
from collections import defaultdict

/Users/zhangwen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class DistMult:
    @property
    def variables(self):
        return self.__variables

    @property
    def num_triple_train(self):
        return self.__num_triple_train

    @property 
    def num_triple_test(self):
        return self.__num_triple_test

    @property
    def testing_data(self):
        return self.__triple_test

    @property 
    def num_entity(self):
        return self.__num_entity

    @property
    def embedding_entity(self):
        return self.__embedding_entity


    @property
    def embedding_relation(self):
        return self.__embedding_relation

    @property
    def hr_t(self):
        return self.__hr_t

    @property 
    def tr_h(self):
        return self.__tr_h
    
    @property
    def entity2id(self):
        return self.__entity2id
    
    @property
    def relation2id(self):
        return self.__relation2id

    @property
    def id2entity(self):
        return self.__id2entity
    
    @property
    def id2relation(self):
        return self.__id2relation

    def training_data_batch(self, batch_size = 512):
        n_triple = len(self.__triple_train)
        rand_idx = np.random.permutation(n_triple)
        start = 0
        while start < n_triple:
            start_t = timeit.default_timer()
            end = min(start+batch_size, n_triple)
            size = end - start 
            train_triple_positive = np.asarray([ self.__triple_train[x] for x in  rand_idx[start:end]])

            num_neg = 1
            train_negative1 = np.repeat(train_triple_positive, num_neg, axis=0)
            train_negative2 = np.repeat(train_triple_positive, num_neg, axis=0)
            train_negative1[:, 0] = np.random.randint(self.__num_entity, size=num_neg*size)
            train_negative2[:, 2] = np.random.randint(self.__num_entity, size=num_neg*size)
            train_triple_negative = np.concatenate((train_negative1, train_negative2), axis=0)

            start = end
            prepare_t = timeit.default_timer()-start_t

            yield train_triple_positive, train_triple_negative, prepare_t


    def __init__(self, data_dir, negative_sampling,learning_rate, 
             batch_size, max_iter, margin, dimension, norm, evaluation_size, regularizer_weight):
        # this part for data prepare
        self.__data_dir=data_dir
        self.__negative_sampling=negative_sampling
        self.__regularizer_weight = regularizer_weight
        self.__norm = norm

        self.__entity2id={}
        self.__id2entity={}
        self.__relation2id={}
        self.__id2relation={}

        self.__triple_train=[] #[(head_id, relation_id, tail_id),...]
        self.__triple_test=[]
        self.__triple_valid=[]
        self.__triple = []

        self.__num_entity=0
        self.__num_relation=0
        self.__num_triple_train=0
        self.__num_triple_test=0
        self.__num_triple_valid=0

        # load all the file: entity2id.txt, relation2id.txt, train.txt, test.txt, valid.txt
        self.load_data()
        print('finish preparing data. ')


        # this part for the model:
        self.__learning_rate = learning_rate
        self.__batch_size = batch_size
        self.__max_iter = max_iter
        self.__margin = margin
        self.__dimension = dimension
        self.__variables= []
        #self.__norm = norm
        self.__evaluation_size = evaluation_size
        bound = 6 / math.sqrt(self.__dimension)
        with tf.device('/cpu'):
            self.__embedding_entity = tf.get_variable('embedding_entity', [self.__num_entity, self.__dimension],
                                                       initializer=tf.random_uniform_initializer(minval=-bound, maxval=bound, seed = 123), dtype=tf.float32)
            self.__embedding_relation = tf.get_variable('embedding_relation', [self.__num_relation, self.__dimension],
                                                         initializer=tf.random_uniform_initializer(minval=-bound, maxval=bound, seed =124), dtype=tf.float32)
            self.__variables.append(self.__embedding_entity)
            self.__variables.append(self.__embedding_relation)
            print('finishing initializing')


    def load_data(self):
        print('loading entity2id.txt ...')
        with open(os.path.join(self.__data_dir, 'entity2id.txt'), encoding='utf-8') as f:
            self.__entity2id = {line.strip().split('\t')[0]: int(line.strip().split('\t')[1]) for line in f.readlines()}
            self.__id2entity = {value:key for key,value in self.__entity2id.items()}

        print('loading reltion2id.txt ...')     
        with open(os.path.join(self.__data_dir,'relation2id.txt'), encoding='utf-8') as f:
            self.__relation2id = {line.strip().split('\t')[0]: int(line.strip().split('\t')[1]) for line in f.readlines()}
            self.__id2relation = {value:key for key, value in self.__relation2id.items()}

        def load_triple(self, triplefile):
            triple_list = [] #[(head_id, relation_id, tail_id),...]
            with open(os.path.join(self.__data_dir, triplefile), encoding='utf-8') as f:
                for line in f.readlines():
                    line_list = line.strip().split('\t')
                    assert len(line_list) == 3
                    headid = self.__entity2id[line_list[0]]
                    relationid = self.__relation2id[line_list[1]]
                    tailid = self.__entity2id[line_list[2]]
                    triple_list.append((headid, relationid, tailid))
                    self.__hr_t[(headid, relationid)].add(tailid)
                    self.__tr_h[(tailid, relationid)].add(headid)
            return triple_list

        self.__hr_t = defaultdict(set)
        self.__tr_h = defaultdict(set)
        self.__triple_train = load_triple(self, 'train.txt')
        self.__triple_test = load_triple(self, 'test.txt')
        self.__triple_valid = load_triple(self, 'valid.txt')
        self.__triple = np.concatenate([self.__triple_train, self.__triple_test, self.__triple_valid], axis = 0 )

        self.__num_relation = len(self.__relation2id)
        self.__num_entity = len(self.__entity2id)
        self.__num_triple_train = len(self.__triple_train)
        self.__num_triple_test = len(self.__triple_test)
        self.__num_triple_valid = len(self.__triple_valid)

        print('entity number: ' + str(self.__num_entity))
        print('relation number: ' + str(self.__num_relation))
        print('training triple number: ' + str(self.__num_triple_train))
        print('testing triple number: ' + str(self.__num_triple_test))
        print('valid triple number: ' + str(self.__num_triple_valid))


        if self.__negative_sampling == 'bern':
            self.__relation_property_head = {x:[] for x in range(self.__num_relation)} #{relation_id:[headid1, headid2,...]}
            self.__relation_property_tail = {x:[] for x in range(self.__num_relation)} #{relation_id:[tailid1, tailid2,...]}
            self.__relation_property = {x:[] for x in range(self.__num_relation)} 
            for t in self.__triple_train:
                #print(t)
                self.__relation_property_head[t[1]].append(t[0])
                self.__relation_property_tail[t[1]].append(t[2])
            #print(self.__relation_property_head[0])
            #print(self.__relation_property_tail[0])
            for x in self.__relation_property_head.keys():
                t = len(set(self.__relation_property_tail[x]))
                h = len(set(self.__relation_property_head[x]))
                self.__relation_property[x] = float(t)/(h+t+0.000000001)
            #self.__relation_property = {x:(len(set(self.__relation_property_tail[x])))/(len(set(self.__relation_property_head[x]))+ len(set(self.__relation_property_tail[x]))) \
            #							 for x in self.__relation_property_head.keys()} # {relation_id: p, ...} 0< num <1, and for relation replace head entity with the property p
        else: 
            print("unif set don't need to calculate hpt and tph")



    def train(self, inputs):
        embedding_relation = self.__embedding_relation
        embedding_entity = self.__embedding_entity

        triple_positive, triple_negative = inputs # triple_positive:(head_id,relation_id,tail_id)

        #norm_entity = tf.nn.l2_normalize(embedding_entity, dim = 1)
        #norm_relation = tf.nn.l2_normalize(embedding_relation, dim = 1)
        norm_entity = embedding_entity
        norm_relation = embedding_relation
        norm_entity_l2sum = tf.sqrt(tf.reduce_sum(norm_entity**2, axis = 1))

        embedding_positive_head = tf.nn.embedding_lookup(norm_entity, triple_positive[:, 0])
        embedding_positive_tail = tf.nn.embedding_lookup(norm_entity, triple_positive[:, 2])
        embedding_positive_relation = tf.nn.embedding_lookup(norm_relation, triple_positive[:, 1])

        embedding_negative_head = tf.nn.embedding_lookup(norm_entity, triple_negative[:, 0])
        embedding_negative_tail = tf.nn.embedding_lookup(norm_entity, triple_negative[:, 2])
        embedding_negative_relation = tf.nn.embedding_lookup(norm_relation, triple_negative[:, 1])

        score_positive = tf.reduce_sum(embedding_positive_head * embedding_positive_relation * embedding_positive_tail, axis = 1)
        score_negative = tf.reduce_sum(embedding_negative_head * embedding_negative_relation * embedding_negative_tail, axis = 1)
        score = tf.concat((-score_positive, score_negative), axis =0)
        
        loss_triple = tf.reduce_mean(tf.nn.softplus(score))
        
        self.__loss_regularizer = loss_regularizer = tf.reduce_sum(tf.abs(self.__embedding_relation)) + tf.reduce_sum(tf.abs(self.__embedding_entity))
        return loss_triple + loss_regularizer*self.__regularizer_weight,  norm_entity_l2sum

    def test(self, inputs):
        embedding_relation = self.__embedding_relation
        embedding_entity = self.__embedding_entity

        triple_test = inputs # (headid, tailid, tailid)
        head_vec = tf.nn.embedding_lookup(embedding_entity, triple_test[0])
        rel_vec = tf.nn.embedding_lookup(embedding_relation, triple_test[1])
        tail_vec = tf.nn.embedding_lookup(embedding_entity, triple_test[2])

        norm_embedding_entity = tf.nn.l2_normalize(embedding_entity, dim =1 )
        norm_embedding_relation = tf.nn.l2_normalize(embedding_relation, dim = 1)
        norm_head_vec = tf.nn.embedding_lookup(norm_embedding_entity, triple_test[0])
        norm_rel_vec = tf.nn.embedding_lookup(norm_embedding_relation, triple_test[1])
        norm_tail_vec = tf.nn.embedding_lookup(norm_embedding_entity, triple_test[2])
        
        _, id_replace_head = tf.nn.top_k(tf.reduce_sum(embedding_entity * rel_vec * tail_vec, axis=1), k=self.__num_entity)
        _, id_replace_tail = tf.nn.top_k(tf.reduce_sum(head_vec * rel_vec * embedding_entity, axis=1), k=self.__num_entity)
        
        _, norm_id_replace_head = tf.nn.top_k(tf.reduce_sum(norm_embedding_entity * norm_rel_vec * norm_tail_vec, axis=1), k=self.__num_entity)
        _, norm_id_replace_tail = tf.nn.top_k(tf.reduce_sum(norm_head_vec * norm_rel_vec * norm_embedding_entity, axis=1), k=self.__num_entity)

        return id_replace_head, id_replace_tail, norm_id_replace_head, norm_id_replace_tail

In [3]:
def train_operation(model, learning_rate=0.01, margin=1.0, optimizer_str = 'gradient'):
    with tf.device('/cpu'):
        train_triple_positive_input = tf.placeholder(tf.int32, [None, 3])
        train_triple_negative_input = tf.placeholder(tf.int32, [None, 3])

        loss, norm_entity = model.train([train_triple_positive_input, train_triple_negative_input])
        if optimizer_str == 'gradient':
            optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
        elif optimizer_str == 'rms':
            optimizer = tf.train.RMSPropOptimizer(learning_rate = learning_rate)
        elif optimizer_str == 'adam':
            optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        else:
            raise NotImplementedError("Dose not support %s optimizer" %optimizer_str)

        grads = optimizer.compute_gradients(loss, model.variables)
        op_train = optimizer.apply_gradients(grads)

        return train_triple_positive_input, train_triple_negative_input, loss, op_train, norm_entity

In [4]:
def test_operation(model):
    with tf.device('/cpu'):
        test_triple = tf.placeholder(tf.int32, [3])
        head_rank, tail_rank, norm_head_rank, norm_tail_rank = model.test(test_triple)
        return test_triple, head_rank, tail_rank, norm_head_rank, norm_tail_rank

In [5]:
class Args:
    pass

In [6]:
# 测试一个样本函数
def test_one_sample(model, trp, session):
    t = trp
    id_replace_head , id_replace_tail, norm_id_replace_head , norm_id_replace_tail  = session.run([head_rank, tail_rank, norm_head_rank, norm_tail_rank], {test_triple:t})
    hr_t = model.hr_t
    tr_h = model.tr_h
    
    hrank = 0
    fhrank = 0
    predicted_head_tmp = []
    for i in range(len(id_replace_head)):
        val = id_replace_head[i]
        predicted_head_tmp.append(val)
        if val == t[0]:
            break
        else: 
            hrank += 1
            fhrank += 1 
            if val in tr_h[(t[2],t[1])]:
                fhrank -= 1
                _ = predicted_head_tmp.pop()
    predicted_head_tmp = [id_replace_head[i] for i in range(len(id_replace_head))]
    
    norm_hrank = 0
    norm_fhrank = 0
    norm_predicted_head_tmp = []
    for i in range(len(norm_id_replace_head)):
        val = norm_id_replace_head[i]
        norm_predicted_head_tmp.append(val)
        if val == t[0]:
            break
        else: 
            norm_hrank += 1
            norm_fhrank += 1 
            if val in tr_h[(t[2],t[1])]:
                norm_fhrank -= 1
                _ = norm_predicted_head_tmp.pop()
    norm_predicted_head_tmp = [id_replace_head[i] for i in range(len(norm_id_replace_head))]

    trank = 0
    ftrank = 0
    predicted_tail_tmp = []
    for i in range(len(id_replace_tail)):
        val = id_replace_tail[i]
        predicted_tail_tmp.append(val)
        if val == t[2]:
            break
        else:
            trank += 1
            ftrank += 1
            if val in hr_t[(t[0], t[1])]:
                ftrank -= 1
                _ = predicted_tail_tmp.pop()
    predicted_tail_tmp = [id_replace_tail[i] for i in range(len(id_replace_tail))]

    norm_trank = 0
    norm_ftrank = 0
    norm_predicted_tail_tmp = []
    for i in range(len(norm_id_replace_tail)):
        val = norm_id_replace_tail[i]
        norm_predicted_tail_tmp.append(val)
        if val == t[2]:
            break
        else:
            norm_trank += 1
            norm_ftrank += 1
            if val in hr_t[(t[0], t[1])]:
                norm_ftrank -= 1
                _ = norm_predicted_tail_tmp.pop()
    norm_predicted_tail_tmp = [id_replace_tail[i] for i in range(len(norm_id_replace_tail))]
    
    return hrank+1, fhrank+1, trank+1, ftrank+1, norm_hrank+1, norm_fhrank+1, norm_trank+1, norm_ftrank+1, \
            predicted_head_tmp, predicted_tail_tmp, norm_predicted_head_tmp, norm_predicted_tail_tmp

In [7]:
def hit(rank_head, rank_tail, k):
    n_test = len(rank_head)
    assert len(rank_head) == len(rank_tail)
    hit_head = np.sum(np.asarray(np.asarray(rank_head)<=k , dtype=np.float32))/n_test
    hit_tail = np.sum(np.asarray(np.asarray(rank_tail)<=k , dtype=np.float32))/n_test
    hit = (hit_head + hit_tail)/2.0
    return hit_head, hit_tail, hit


In [8]:
def test(n_test):
    predicted_tail = []
    norm_predicted_tail = []
    predicted_head = []
    norm_predicted_head = []
    np.random.shuffle(testing_data)
    for i in range(n_test):
        print('[%.2f sec] --- testing[%d/%d]' %(timeit.default_timer()-start, i+1, n_test), end='\r')
        t = testing_data[i]
        hrank, fhrank, trank, ftrank, norm_hrank, norm_fhrank, norm_trank, norm_ftrank, \
                predicted_head_tmp, predicted_tail_tmp, norm_predicted_head_tmp, norm_predicted_tail_tmp = test_one_sample(model, t, session)
        #print(hrank, fhrank, trank, ftrank, norm_hrank, norm_fhrank, norm_trank, norm_ftrank)
        rank_head.append(hrank)
        rank_tail.append(trank)
        filter_rank_head.append(fhrank)
        filter_rank_tail.append(ftrank)
        
        norm_rank_head.append(norm_hrank)
        norm_rank_tail.append(norm_trank)
        norm_filter_rank_head.append(norm_fhrank)
        norm_filter_rank_tail.append(norm_ftrank)

        predicted_tail.append(predicted_tail_tmp)
        norm_predicted_tail.append(norm_predicted_tail_tmp)
        predicted_head.append(predicted_head_tmp)
        norm_predicted_head.append(norm_predicted_head_tmp)
    mean_rank_head = np.sum(rank_head, dtype=np.float32)/n_test
    mean_rank_tail = np.sum(rank_tail, dtype=np.float32)/n_test
    filter_mean_rank_head = np.sum(filter_rank_head, dtype=np.float32)/n_test
    filter_mean_rank_tail = np.sum(filter_rank_tail, dtype=np.float32)/n_test

    norm_mean_rank_head = np.sum(norm_rank_head, dtype=np.float32)/n_test
    norm_mean_rank_tail = np.sum(norm_rank_tail, dtype=np.float32)/n_test
    norm_filter_mean_rank_head = np.sum(norm_filter_rank_head, dtype=np.float32)/n_test
    norm_filter_mean_rank_tail = np.sum(norm_filter_rank_tail, dtype=np.float32)/n_test

    mean_reciprocal_rank_head = np.sum(1.0/np.asarray(rank_head, dtype=np.float32))/n_test
    mean_reciprocal_rank_tail = np.sum(1.0/np.asarray(rank_tail, dtype=np.float32))/n_test
    filter_mean_reciprocal_rank_head = np.sum(1.0/np.asarray(filter_rank_head, dtype=np.float32))/n_test
    filter_mean_reciprocal_rank_tail = np.sum(1.0/np.asarray(filter_rank_tail, dtype=np.float32))/n_test

    hit1_head, hit1_tail, hit1 = hit(rank_head, rank_tail, 1)
    filter_hit1_head, filter_hit1_tail, filter_hit1 = hit(filter_rank_head, filter_rank_tail, 1)
    hit3_head, hit3_tail, hit3 = hit(rank_head, rank_tail, 3)
    filter_hit3_head, filter_hit3_tail, filter_hit3 = hit(filter_rank_head, filter_rank_tail, 3)
    hit10_head, hit10_tail, hit10 = hit(rank_head, rank_tail, 10)
    filter_hit10_head, filter_hit10_tail, filter_hit10 = hit(filter_rank_head, filter_rank_tail, 10)


    print('iter:%d --MR: %.2f  --MRR: %.2f  --hit@1: %.2f   --hit@3: %.2f    --hit@10: %.2f' %(n_iter, (mean_rank_head+ mean_rank_tail)/2, 
                                                            (mean_reciprocal_rank_head + mean_reciprocal_rank_tail)/2, 
                                                            hit1, hit3, hit3))
    print('iter:%d --FMR: %.2f --FMRR: %.2f --Fhit@1: %.2f  --Fhit@3: %.2f   --Fhit@10: %.2f' %(n_iter, (filter_mean_rank_head+ filter_mean_rank_tail)/2, 
                                                                    (filter_mean_reciprocal_rank_head + filter_mean_reciprocal_rank_tail)/2,
                                                                    filter_hit1, filter_hit3, filter_hit10)) 
    

In [9]:
# 设置参数等
args  = Args()
args.data_dir = './data/FB15k-237/'
args.learning_rate = 0.005
args.batch_size = 2048
args.max_iter = 200
args.optimizer = 'adam'
args.dimension = 300
args.margin = 3
args.norm = 'L2'
args.evaluation_size = 500
args.save_dir = 'output/'
args.negative_sampling = 'bern'
args.evaluate_per_iteration = 1
args.evaluate_worker = 3
args.regularizer_weight = 1e-7
args.n_test = 100
args.save_per = 100
args.n_worker = 5
args.max_iter = 50

print(args)
model = DistMult(negative_sampling=args.negative_sampling, data_dir=args.data_dir,
                learning_rate=args.learning_rate, batch_size=args.batch_size,
                max_iter=args.max_iter, margin=args.margin, 
                dimension=args.dimension, norm=args.norm, evaluation_size=args.evaluation_size, 
                regularizer_weight = args.regularizer_weight)

train_triple_positive_input, train_triple_negative_input, loss, op_train, norm_entity = train_operation(model, learning_rate = args.learning_rate, margin = args.margin, optimizer_str = args.optimizer)
test_triple, head_rank, tail_rank , norm_head_rank, norm_tail_rank= test_operation(model)

loading entity2id.txt ...
loading reltion2id.txt ...
entity number: 14541
relation number: 237
training triple number: 272115
testing triple number: 20466
valid triple number: 17535
finish preparing data. 
Instructions for updating:
Colocations handled automatically by placer.
finishing initializing
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
dim is deprecated, use axis instead


In [ ]:
# 训练模型
config = tf.ConfigProto()
config.gpu_options.allow_growth = False
config.log_device_placement = False
config.allow_soft_placement = True
config.gpu_options.per_process_gpu_memory_fraction=0.68
session = tf.Session(config=config)
session.as_default()

tf.initialize_all_variables().run(session=session)
saver = tf.train.Saver()


for n_iter in range(args.max_iter):
    accu_loss =0.
    batch = 0
    num_batch = model.num_triple_train/args.batch_size
    start_time = timeit.default_timer()
    prepare_time = 0.

    for tp, tn , t in  model.training_data_batch(batch_size= args.batch_size):
        l, _, norm_e = session.run([loss, op_train, norm_entity], {train_triple_positive_input:tp, train_triple_negative_input: tn})
        accu_loss += l
        batch += 1
        print('[%.2f sec](%d/%d): -- loss: %.5f' %(timeit.default_timer()-start_time, batch, num_batch , l), end='\r')
        prepare_time += t
    if n_iter%1 == 0:
        print('iter[%d] ---loss: %.5f ---time: %.2f ---prepare time : %.2f' %(n_iter, accu_loss, timeit.default_timer()-start_time, prepare_time))

    if n_iter % args.save_per == 0 or n_iter ==0 or n_iter == args.max_iter-1:
        save_path = saver.save(session, os.path.join('./save/DistMult_' + str(n_iter) + '.ckpt'))
        print('Model saved at %s' % save_path)

    if n_iter %args.evaluate_per_iteration == 0 or n_iter ==0 or n_iter == args.max_iter-1:
        rank_head = []
        rank_tail = []
        filter_rank_head = []
        filter_rank_tail = []

        norm_rank_head = []
        norm_rank_tail = []
        norm_filter_rank_head = []
        norm_filter_rank_tail = []

        start = timeit.default_timer()
        testing_data = model.testing_data
        hr_t = model.hr_t
        tr_h = model.tr_h
        n_test = args.n_test
        test(n_test)


Instructions for updating:
Use `tf.global_variables_initializer` instead.
iter[0] ---loss: 101.43665 ---time: 40.85 ---prepare time : 0.31
Model saved at ./save/DistMult_0.ckpt
iter:0 --MR: 5960.71  --MRR: 0.01  --hit@1: 0.01   --hit@3: 0.01    --hit@10: 0.01
iter:0 --FMR: 5852.61 --FMRR: 0.01 --Fhit@1: 0.01  --Fhit@3: 0.01   --Fhit@10: 0.01
iter[1] ---loss: 93.67135 ---time: 39.79 ---prepare time : 0.31
iter:1 --MR: 2718.01  --MRR: 0.06  --hit@1: 0.02   --hit@3: 0.07    --hit@10: 0.07
iter:1 --FMR: 2562.93 --FMRR: 0.13 --Fhit@1: 0.07  --Fhit@3: 0.16   --Fhit@10: 0.24
iter[2] ---loss: 78.94437 ---time: 49.94 ---prepare time : 0.41
iter:2 --MR: 2096.07  --MRR: 0.08  --hit@1: 0.04   --hit@3: 0.07    --hit@10: 0.07
iter:2 --FMR: 1926.93 --FMRR: 0.13 --Fhit@1: 0.08  --Fhit@3: 0.15   --Fhit@10: 0.23
iter[3] ---loss: 49.86318 ---time: 41.80 ---prepare time : 0.35
iter:3 --MR: 1324.98  --MRR: 0.05  --hit@1: 0.01   --hit@3: 0.04    --hit@10: 0.04
iter:3 --FMR: 1133.86 --FMRR: 0.13 --Fhit@1: 0.

iter[34] ---loss: 13.27316 ---time: 47.34 ---prepare time : 0.42
iter:34 --MR: 608.02  --MRR: 0.14  --hit@1: 0.06   --hit@3: 0.17    --hit@10: 0.17
iter:34 --FMR: 419.53 --FMRR: 0.26 --Fhit@1: 0.16  --Fhit@3: 0.28   --Fhit@10: 0.45
iter[35] ---loss: 13.26994 ---time: 46.48 ---prepare time : 0.43
iter:35 --MR: 753.16  --MRR: 0.13  --hit@1: 0.07   --hit@3: 0.14    --hit@10: 0.14
iter:35 --FMR: 514.03 --FMRR: 0.24 --Fhit@1: 0.17  --Fhit@3: 0.23   --Fhit@10: 0.45
iter[36] ---loss: 13.07923 ---time: 44.25 ---prepare time : 0.38
iter:36 --MR: 1049.81  --MRR: 0.11  --hit@1: 0.05   --hit@3: 0.12    --hit@10: 0.12
iter:36 --FMR: 856.95 --FMRR: 0.19 --Fhit@1: 0.11  --Fhit@3: 0.22   --Fhit@10: 0.30
iter[37] ---loss: 13.03575 ---time: 44.58 ---prepare time : 0.41
iter:37 --MR: 945.20  --MRR: 0.12  --hit@1: 0.06   --hit@3: 0.12    --hit@10: 0.12
iter:37 --FMR: 600.32 --FMRR: 0.18 --Fhit@1: 0.09  --Fhit@3: 0.21   --Fhit@10: 0.39
iter[38] ---loss: 13.01621 ---time: 41.04 ---prepare time : 0.33
iter:3

In [ ]:
# 测试模型
predicted_tail = []
norm_predicted_tail = []
predicted_head = []
norm_predicted_head = []

rank_head = []
rank_tail = []
filter_rank_head = []
filter_rank_tail = []

norm_rank_head = []
norm_rank_tail = []
norm_filter_rank_head = []
norm_filter_rank_tail = []

start = timeit.default_timer()
testing_data = model.testing_data
# hr_t = model.hr_t
# tr_h = model.tr_h
n_test = args.n_test
if n_iter == args.max_iter-1:	n_test = model.num_triple_test
predicted_tail = []
norm_predicted_tail = []
predicted_head = []
norm_predicted_head = []
test(n_test)

我们按照设置的参数跑了40个iteration，可以看到DistMult在FB15k-237的数据集上已经有了明显的预测效果。
本demo中不包括调参的部分，有兴趣的同学可以阅读原文并自行尝试不同的参数组合，并观察对模型训练和预测结果的影响 :-)